In [1]:
import time
import pandas as pd
from datetime import datetime
from appium import webdriver
from appium.options.android import UiAutomator2Options
from appium.webdriver.common.appiumby import AppiumBy
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# ===================== [설정 영역] =====================
APP_PACKAGE = "kr.go.minwon.m"
APP_ACTIVITY = "kr.go.minwon.m.BrowserActivity"
DEVICE_NAME = "Galaxy S24"
APPIUM_SERVER_URL = "http://127.0.0.1:4723"

# 반복 횟수
REPEAT_COUNT = 10

# 검색어 입력값
KEYWORD = "여권"

In [3]:
# 메인 테스트
def run_gov24_search_test():
    options = UiAutomator2Options()
    options.device_name = DEVICE_NAME
    options.app_package = APP_PACKAGE
    options.app_activity = APP_ACTIVITY
    options.automation_name = "UiAutomator2"
    options.no_reset = True
    options.new_command_timeout = 300
    
    # 키보드 관련 설정 (입력 안정성)
    options.set_capability("connectHardwareKeyboard", True)

    print(f"--- 정부24 검색 성능 측정 ({REPEAT_COUNT}회) 시작 ---")
    
    driver = None
    results = []

    try:
        driver = webdriver.Remote(APPIUM_SERVER_URL, options=options)
        wait = WebDriverWait(driver, 20)

        print("📱 앱 실행 및 메인 화면 대기 중...")
        # 메인 검색창이 뜰 때까지 대기
        wait.until(EC.visibility_of_element_located(
            (AppiumBy.XPATH, '//android.widget.EditText[@resource-id="mainSearch"]')
        ))

        # ===================== 반복 측정 루프 =====================
        for i in range(1, REPEAT_COUNT + 1):
            try:
                print(f"\n[Running] {i}/{REPEAT_COUNT}회차 측정 진행 중...")

                # 1. 메인 검색창 찾기 및 검색어 입력
                search_input = wait.until(EC.visibility_of_element_located(
                    (AppiumBy.XPATH, '//android.widget.EditText[@resource-id="mainSearch"]')
                ))
                
                search_input.click() 
                search_input.clear()
                search_input.send_keys(KEYWORD)
                
                # 2. 검색 버튼 찾기 (wait 추가하여 안정성 확보)
                search_btn = wait.until(EC.element_to_be_clickable(
                    (AppiumBy.XPATH, '//android.widget.Button[@text="검색"]')
                ))
                
                # [Time Start]
                start_time = time.time() 
                search_btn.click()
                
                # 3. 결과 화면 검증 (검색 결과 텍스트 대기)
                wait.until(EC.presence_of_element_located(
                    (AppiumBy.XPATH, '//android.widget.TextView[@text="검색 결과"]')
                ))
                
                # [Time End]
                end_time = time.time()
                
                duration = end_time - start_time
                print(f"⏱️ {i}회차 소요 시간: {duration:.4f}초")

                results.append({
                    "회차": i,
                    "측정시각": datetime.now().strftime("%H:%M:%S"),
                    "검색어": KEYWORD,
                    "검색반응속도(초)": round(duration, 4)
                })

                # ==========================================================
                # [수정됨] UI 버튼 대신 하드웨어 '뒤로 가기' 키 사용
                # ==========================================================
                print("🔙 하드웨어 뒤로가기 키 입력")
                driver.press_keycode(4) # Android KeyCode 4 = Back Button

                # 5. 메인 화면 로딩 대기 (다음 루프 준비)
                wait.until(EC.visibility_of_element_located(
                    (AppiumBy.XPATH, '//android.widget.EditText[@resource-id="mainSearch"]')
                ))
                
                time.sleep(1) # 안정성을 위한 짧은 대기

            except Exception as e:
                print(f"❌ {i}회차 실행 중 에러 발생: {e}")
                # 에러 발생 시에도 뒤로가기를 눌러 복구 시도
                try:
                    driver.press_keycode(4)
                except:
                    pass

        # ===================== CSV 저장 =====================
        print("\n" + "=" * 50)
        print("💾 결과 저장 중...")

        if results:
            df = pd.DataFrame(results)
            speeds = df["검색반응속도(초)"]
            
            # 요약 행 추가
            summary = {
                "회차": "요약",
                "검색반응속도(초)": "",
                "평균(초)": round(speeds.mean(), 4),
                "최소(초)": round(speeds.min(), 4),
                "최대(초)": round(speeds.max(), 4),
                "표준편차(초)": round(speeds.std(ddof=1), 4) if len(speeds) >= 2 else 0.0
            }
            
            df = pd.concat([df, pd.DataFrame([summary])], ignore_index=True)
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            file_name = f"gov24_search_perf_{REPEAT_COUNT}runs_{timestamp}.csv"
            df.to_csv(file_name, index=False, encoding="utf-8-sig")

            print(f"✅ 저장 완료: {file_name}")
            print(df)
        else:
            print("ℹ️ 저장할 데이터가 없습니다.")

    except Exception as e:
        print(f"⛔ 치명적 오류: {e}")

    finally:
        if driver:
            driver.quit()

In [4]:
# 테스트 실행
if __name__ == "__main__":
    run_gov24_search_test()

--- 정부24 검색 성능 측정 (10회) 시작 ---
📱 앱 실행 및 메인 화면 대기 중...

[Running] 1/10회차 측정 진행 중...
⏱️ 1회차 소요 시간: 0.8932초
🔙 하드웨어 뒤로가기 키 입력

[Running] 2/10회차 측정 진행 중...
⏱️ 2회차 소요 시간: 0.9003초
🔙 하드웨어 뒤로가기 키 입력

[Running] 3/10회차 측정 진행 중...
⏱️ 3회차 소요 시간: 0.9019초
🔙 하드웨어 뒤로가기 키 입력

[Running] 4/10회차 측정 진행 중...
⏱️ 4회차 소요 시간: 0.8958초
🔙 하드웨어 뒤로가기 키 입력

[Running] 5/10회차 측정 진행 중...
⏱️ 5회차 소요 시간: 0.8942초
🔙 하드웨어 뒤로가기 키 입력

[Running] 6/10회차 측정 진행 중...
⏱️ 6회차 소요 시간: 0.9006초
🔙 하드웨어 뒤로가기 키 입력

[Running] 7/10회차 측정 진행 중...
⏱️ 7회차 소요 시간: 0.8964초
🔙 하드웨어 뒤로가기 키 입력

[Running] 8/10회차 측정 진행 중...
⏱️ 8회차 소요 시간: 0.8881초
🔙 하드웨어 뒤로가기 키 입력

[Running] 9/10회차 측정 진행 중...
⏱️ 9회차 소요 시간: 0.9109초
🔙 하드웨어 뒤로가기 키 입력

[Running] 10/10회차 측정 진행 중...
⏱️ 10회차 소요 시간: 0.9068초
🔙 하드웨어 뒤로가기 키 입력

💾 결과 저장 중...
✅ 저장 완료: gov24_search_perf_10runs_20251204_145009.csv
    회차      측정시각  검색어 검색반응속도(초)   평균(초)   최소(초)   최대(초)  표준편차(초)
0    1  14:49:33   여권    0.8932     NaN     NaN     NaN      NaN
1    2  14:49:37   여권    0.9003     NaN     NaN     NaN      NaN
2